In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import scipy.stats as ss
from pandas_datareader import DataReader
from datetime import datetime

# Make plots larger
plt.rcParams['figure.figsize'] = (15, 9)

In [6]:
facebook = DataReader('FB',  'yahoo', datetime(2016,4,1), datetime(2019,9,1));
facebook.reset_index(inplace=True,drop=False)
facebook.set_index('Date')
facebook['CloseYest'] = facebook['Close'].shift(-1)
facebook['OpenTmr'] = facebook['Open'].shift(-1)
facebook['Close2dayBefore'] = facebook['Close'].shift(-2)
facebook.head()
facebook = facebook.dropna(0)

In [7]:
facebook.tail()

,Date,High,Low,Open,Close,Volume,Adj Close,CloseYest,OpenTmr,Close2dayBefore
854,2019-08-22,184.110001,179.910004,183.429993,182.039993,10821400,182.039993,177.750000,180.839996,180.360001
855,2019-08-23,183.130005,176.660004,180.839996,177.750000,17323400,177.750000,180.360001,179.399994,181.300003
856,2019-08-26,180.500000,178.240005,179.399994,180.360001,8773600,180.360001,181.300003,181.929993,181.759995
857,2019-08-27,184.039993,181.009995,181.929993,181.300003,14399600,181.300003,181.759995,180.529999,185.570007
858,2019-08-28,181.949997,178.919998,180.529999,181.759995,9386100,181.759995,185.570007,183.770004,185.669998


In [8]:
def AR_bootstrap():
    N = facebook.shape[0] # total num days
    num_boot = 1# total num bootstrap
    T= 250 # start day
    window = 200 # training period window
    Y =facebook['OpenTmr'][-(N-(T+1)):].values
    X = facebook[['Close','CloseYest','Close2dayBefore']][-(N-(T+1)):].values
    T = X.shape[0]
#     print('T is: ')
#     print(T)
    X = np.column_stack([np.ones((T,1)),X])
    N = X.shape
    invXX = np.linalg.inv(X.transpose()@X)
    
    beta_hat = invXX@X.transpose()@Y
    #print(beta_hat.shape)
    y_hat = X@beta_hat+beta_hat[0]
    
    'Residuals from OLS'
    residuals = Y - y_hat
    'Variance of Residuals'
    sigma2 = (1/T)*residuals.transpose()@residuals
    'standard deviation of Y or residuals'
    sigma = np.sqrt(sigma2)
    'variance-covariance matrix of beta_hat'
    varcov_beta_hat = (sigma2)*invXX
    std_beta_hat = np.sqrt(T*np.diag(varcov_beta_hat))

    'Calculate R-square'
    R_square = 1 - (residuals.transpose()@residuals)/(T*np.var(Y))
    adj_R_square = 1-(1-R_square)*(T-1)/(T-N[1])

    'Test Each Coefficient: beta_i'
    'Null Hypothesis: beta_i = 0'
    t_stat = (beta_hat.transpose()-0)/std_beta_hat

    p_val = 1-ss.norm.cdf(t_stat)
    rmse_ar1 = np.sqrt(np.mean((Y- y_hat)**2))
    return y_hat,rmse_ar1


In [ ]:
# def AR_bootstrap1(X,Y):
#     T = 250
#     num_boost=5
#     #print(T)
#     N = X.shape[0]
#     #print(N)
    
#     yhat_ema = np.zeros(N-(T+1))
#     window = 200
#     num_boost = 300 # increase
    
#     for t in range(T+1,N-2):
#         X_train = X[t-window:t-1]
#         #X_train = np.column_stack([np.ones((len(X_train),1)),X_train])
#         Y_train = Y[t-window:t-1]
#         #print(X_train.shape)
#         #print(Y_train.shape)
#         X_pred = X[t-1:t]
#         X_pred = np.column_stack([np.ones((len(X_pred),1)),X_pred])
        
#         yhat_train = AR_bootstrap(X_train , Y_train)[1]
#         res_train = Y_train - yhat_train
       
#         y_pred_all = np.zeros(num_boost)
       
#         for i in range (0,num_boost):
#             #err = np.random.choice(res_train,(window-1, ),replace = True)
#             err = res_train
#             y_bstr = yhat_train + err
#             beta_bstr = AR_bootstrap(X_train,y_bstr)[0]
#             #print(X_pred.shape)
#             #print(beta_bstr.shape)
 
#             y_pred_bstr = X_pred@beta_bstr
#             y_pred_all[i] = y_pred_bstr
           
#         y_pred_ema = y_pred_all.mean()
#         yhat_ema[t-(T+1)] = y_pred_ema
   
#     #rmse_ema = np.sqrt(np.mean((Y[:-1] - yhat_ema[:-1])**2))
   
#     return yhat_train#,rmse_ema

In [9]:
y,rmse=AR_bootstrap()
print(rmse)

1.9076290240331346


## Professionalism
50% Minghao Ru

50% Tong Yang


## Licensing
Copyright <2020> Minghao Ru, Tong Yang

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.